In [1]:
import pandas as pd
import numpy as np

In [2]:
TY_label = pd.read_excel('../../data/TY_training_labels_v2.xlsx')
TY_data = pd.read_csv('../../data/TY_training_ULT.csv')

/var/folders/yp/_xstfn5s2nb1lmynh4_4ghmc0000gn/T/ipykernel_36915/879553019.py:2: DtypeWarning: Columns (1449,1450,1470,1471,1472,1473,1474,1475,1476,1477,1478,1479,1480,1481,1482,1483,1484,1485,1486,1487,1488) have mixed types. Specify dtype option on import or set low_memory=False.
  TY_data = pd.read_csv('../../data/TY_training_ULT.csv')


# Drop non-test columns

In [3]:
# Function to drop columns (tests) in the raw data file which are NOT tests (i.e metadata like start time, lot number etc)
def test_only(datafile,labelfile):
    keys = list(labelfile['Name'])
    df = datafile[keys]
    return df

In [4]:
#Applying the functions to file 
TY_dropped = test_only(TY_data,TY_label)
#Checking the shape and how the data looks like
print(TY_dropped.shape)
print(TY_dropped.head())

(79783, 1402)
   temperature  Device_Test_Time  Part_Id  a_openXv_SIG_x_x_500uA__NFC_GPIO0  \
0        125.0           71734.0      1.0                          -0.544749   
1        125.0           71734.0      2.0                          -0.542050   
2        125.0           71734.0      3.0                          -0.540997   
3        125.0           71734.0      4.0                          -0.538849   
4        125.0           71734.0      5.0                          -0.536129   

   a_shortXv_SIG_x_x_500uA__NFC_GPIO0  a_openXv_SIG_x_x_500uA__NFC_GPIO1  \
0                           -0.544749                          -0.543823   
1                           -0.542050                          -0.543497   
2                           -0.540997                          -0.540728   
3                           -0.538849                          -0.538906   
4                           -0.536129                          -0.537837   

   a_shortXv_SIG_x_x_500uA__NFC_GPIO1  a_openXv_

# Drop unlabelled columns (without target variable)

In [5]:
#Function to find out non applicable rows (no distribution, not part of training data)
def non_applicable (df_label):
    null_rowsLabel = df_label[df_label['Distribution Type'].isnull()]
    null_rowsLabel = list(null_rowsLabel['Name'])
    return null_rowsLabel

In [6]:
#Applying the above function to set of data
TY_non_applicable = non_applicable(TY_label)
#Printing the tests (columns) to see which tests do not have a distribution (i.e not applicable)
print(TY_non_applicable)
print(len(TY_non_applicable))

[]
0


In [7]:
#Function to faciliate the dropping of the null rows in labels file
def drop_null_labels(label_df,not_applicable):
    df = label_df.drop(label_df[label_df.Name.isin(not_applicable)].index)
    return df

In [8]:
# Applying it to set of data
TY_label_final = drop_null_labels(TY_label,TY_non_applicable)
TY_label_final

,Name,Mean,Standard Deviation,Parameter Type,Distribution Type,Comments
0,temperature,125.000000,0.000000,NaN,functional,NaN
1,Device_Test_Time,52517.820212,1812.012713,PARAMETRIC,bimodal,NaN
2,Part_Id,2849.892872,1645.108043,PARAMETRIC,bimodal,NaN
3,a_openXv_SIG_x_x_500uA__NFC_GPIO0,-0.533726,0.013324,PARAMETRIC,outlier,NaN
4,a_shortXv_SIG_x_x_500uA__NFC_GPIO0,-0.533726,0.013324,PARAMETRIC,outlier,NaN
...,...,...,...,...,...,...
1397,f_nvm_programXxXPgmPatrepairinfo2M1YvhsYvhsYyY...,-0.974299,0.158303,PARAMETRIC,functional,NaN
1398,f_nvm_readifrXxXVfyPatrepairinfo2M1YvhsYvhsYyY...,-0.974299,0.158303,PARAMETRIC,functional,NaN
1399,f_nvm_XxXRdoutPatrepairinfo3M1YvhsYvhsYyYYy_1....,0.000000,0.000000,PARAMETRIC,functional,NaN
1400,f_nvm_readifrXxXVfyPatrepairinfo3M1YvhsYvhsYyY...,-0.413170,0.492482,PARAMETRIC,functional,NaN


# Filter columns down to test name and distribution 

In [9]:
TY_target = TY_label_final[['Name','Distribution Type']]
TY_target

,Name,Distribution Type
0,temperature,functional
1,Device_Test_Time,bimodal
2,Part_Id,bimodal
3,a_openXv_SIG_x_x_500uA__NFC_GPIO0,outlier
4,a_shortXv_SIG_x_x_500uA__NFC_GPIO0,outlier
...,...,...
1397,f_nvm_programXxXPgmPatrepairinfo2M1YvhsYvhsYyY...,functional
1398,f_nvm_readifrXxXVfyPatrepairinfo2M1YvhsYvhsYyY...,functional
1399,f_nvm_XxXRdoutPatrepairinfo3M1YvhsYvhsYyYYy_1....,functional
1400,f_nvm_readifrXxXVfyPatrepairinfo3M1YvhsYvhsYyY...,functional


In [10]:
distribution_counts = TY_target['Distribution Type'].value_counts()
distribution_counts

Distribution Type
functional    1063
outlier         91
longtail        72
normal          69
discrete        54
bimodal         53
Name: count, dtype: int64

In [11]:
TY_transposed = TY_dropped.T.reset_index(inplace=False, drop=False)

TY_transposed.columns = ['Name'] + [i for i in range(1, len(TY_transposed.columns))]
TY_transposed

,Name,1,2,3,4,5,6,7,8,9,...,79774,79775,79776,79777,79778,79779,79780,79781,79782,79783
0,temperature,125.000000,125.00000,125.000000,125.000000,125.000000,125.000000,125.000000,125.000000,125.000000,...,125.000000,125.000000,125.000000,125.000000,125.000000,125.000000,125.000000,125.000000,125.000000,125.00000
1,Device_Test_Time,71734.000000,71734.00000,71734.000000,71734.000000,71734.000000,71734.000000,71734.000000,71734.000000,71734.000000,...,50853.000000,50853.000000,50853.000000,50853.000000,50853.000000,50853.000000,50853.000000,50853.000000,50853.000000,50853.00000
2,Part_Id,1.000000,2.00000,3.000000,4.000000,5.000000,6.000000,7.000000,8.000000,9.000000,...,5689.000000,5690.000000,5691.000000,5692.000000,5693.000000,5694.000000,5695.000000,5696.000000,5697.000000,5698.00000
3,a_openXv_SIG_x_x_500uA__NFC_GPIO0,-0.544749,-0.54205,-0.540997,-0.538849,-0.536129,-0.535918,-0.534637,-0.534009,-0.544783,...,-0.535343,-0.535356,-0.536438,-0.536239,-0.541173,-0.541914,-0.539849,-0.537799,-0.537331,-0.53763
4,a_shortXv_SIG_x_x_500uA__NFC_GPIO0,-0.544749,-0.54205,-0.540997,-0.538849,-0.536129,-0.535918,-0.534637,-0.534009,-0.544783,...,-0.535343,-0.535356,-0.536438,-0.536239,-0.541173,-0.541914,-0.539849,-0.537799,-0.537331,-0.53763
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1397,f_nvm_programXxXPgmPatrepairinfo2M1YvhsYvhsYyY...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1398,f_nvm_readifrXxXVfyPatrepairinfo2M1YvhsYvhsYyY...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1399,f_nvm_XxXRdoutPatrepairinfo3M1YvhsYvhsYyYYy_1....,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1400,f_nvm_readifrXxXVfyPatrepairinfo3M1YvhsYvhsYyY...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
ty_merged = pd.merge(TY_target, TY_transposed, on='Name', how='inner')
df_ty = ty_merged.T

In [13]:
# extract out the features 
features_ty = df_ty.iloc[2:]
features_ty

,0,1,2,3,4,5,6,7,8,9,...,1392,1393,1394,1395,1396,1397,1398,1399,1400,1401
1,125.0,71734.0,1.0,-0.544749,-0.544749,-0.543823,-0.543823,-0.421532,-0.421532,-0.422727,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,125.0,71734.0,2.0,-0.54205,-0.54205,-0.543497,-0.543497,-0.417037,-0.417037,-0.417872,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,125.0,71734.0,3.0,-0.540997,-0.540997,-0.540728,-0.540728,-0.41555,-0.41555,-0.416643,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,125.0,71734.0,4.0,-0.538849,-0.538849,-0.538906,-0.538906,-0.412185,-0.412185,-0.413197,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,125.0,71734.0,5.0,-0.536129,-0.536129,-0.537837,-0.537837,-0.408527,-0.408527,-0.411046,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79779,125.0,50853.0,5694.0,-0.541914,-0.541914,-0.541302,-0.541302,-0.419085,-0.419085,-0.418765,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79780,125.0,50853.0,5695.0,-0.539849,-0.539849,-0.540376,-0.540376,-0.416837,-0.416837,-0.417859,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79781,125.0,50853.0,5696.0,-0.537799,-0.537799,-0.537329,-0.537329,-0.410395,-0.410395,-0.411985,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79782,125.0,50853.0,5697.0,-0.537331,-0.537331,-0.538301,-0.538301,-0.412194,-0.412194,-0.410885,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# extract out the target labels
labels_ty = df_ty.iloc[:2]
labels_ty

,0,1,2,3,4,5,6,7,8,9,...,1392,1393,1394,1395,1396,1397,1398,1399,1400,1401
Name,temperature,Device_Test_Time,Part_Id,a_openXv_SIG_x_x_500uA__NFC_GPIO0,a_shortXv_SIG_x_x_500uA__NFC_GPIO0,a_openXv_SIG_x_x_500uA__NFC_GPIO1,a_shortXv_SIG_x_x_500uA__NFC_GPIO1,a_openXv_SIG_x_x_500uA__NFC-GPIO2_AO,a_shortXv_SIG_x_x_500uA__NFC-GPIO2_AO,a_openXv_SIG_x_x_500uA__NFC_GPIO3_AO,...,f_nvm_readifrXxXVfyPatrepairinfo1M1YvhsYvhsYyY...,f_nvm_programXxXPgmPatrepairinfo1M1YvhsYvhsYyY...,f_nvm_readifrXxXVfyPatrepairinfo1M1YvhsYvhsYyY...,f_nvm_XxXRdoutPatrepairinfo2M1YvhsYvhsYyYYy_1....,f_nvm_readifrXxXVfyPatrepairinfo2M1YvhsYvhsYyY...,f_nvm_programXxXPgmPatrepairinfo2M1YvhsYvhsYyY...,f_nvm_readifrXxXVfyPatrepairinfo2M1YvhsYvhsYyY...,f_nvm_XxXRdoutPatrepairinfo3M1YvhsYvhsYyYYy_1....,f_nvm_readifrXxXVfyPatrepairinfo3M1YvhsYvhsYyY...,f_nvm_programXxXPgmPatrepairinfo3M1YvhsYvhsYyY...
Distribution Type,functional,bimodal,bimodal,outlier,outlier,outlier,outlier,outlier,outlier,outlier,...,functional,functional,functional,functional,functional,functional,functional,functional,functional,functional


# Scaling features by standardizing

In [15]:
# standardize the numerical features
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
normalized_data = scaler.fit_transform(features_ty)

scaled_features_ty = pd.DataFrame(normalized_data) # convert to pandas dataframe
scaled_features_ty

,0,1,2,3,4,5,6,7,8,9,...,1392,1393,1394,1395,1396,1397,1398,1399,1400,1401
0,0.0,10.604948,-1.731747,-0.827373,-0.827373,-0.741794,-0.741794,-0.955227,-0.955227,-1.032775,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,10.604948,-1.731139,-0.624798,-0.624798,-0.717147,-0.717147,-0.605865,-0.605865,-0.636752,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0,10.604948,-1.730531,-0.545780,-0.545780,-0.507799,-0.507799,-0.490336,-0.490336,-0.536527,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.0,10.604948,-1.729923,-0.384546,-0.384546,-0.370025,-0.370025,-0.228798,-0.228798,-0.255459,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.0,10.604948,-1.729315,-0.180372,-0.180372,-0.289197,-0.289197,0.055479,0.055479,-0.079966,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79778,0.0,-0.918774,1.728838,-0.614568,-0.614568,-0.551158,-0.551158,-0.765015,-0.765015,-0.709572,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79779,0.0,-0.918774,1.729446,-0.459564,-0.459564,-0.481179,-0.481179,-0.590377,-0.590377,-0.635675,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79780,0.0,-0.918774,1.730054,-0.305768,-0.305768,-0.250775,-0.250775,-0.089698,-0.089698,-0.156604,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79781,0.0,-0.918774,1.730661,-0.270574,-0.270574,-0.324300,-0.324300,-0.229536,-0.229536,-0.066892,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Feature engineering

In [16]:
from scipy.stats import skew, kurtosis, norm, kstest, zscore, shapiro
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
import warnings

def all_features(df):
    feature_vectors = []
    for column in df.columns:        
        values = df[column].dropna()
        if values.empty:
            return pd.DataFrame() # return empty features if no values 
        # statistical properties
        mean = values.mean()
        median = values.median()
        std_dev = values.std()
        iqr = values.quantile(0.75) - values.quantile(0.25)
        unique_values = np.unique(values)
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", category=RuntimeWarning)
            try:
                if len(unique_values) <= 2:
                    skewness = 0
                    kurt = 0
                else:
                    skewness = skew(values)
                    kurt = kurtosis(values)
            except RuntimeWarning:
                skewness = np.nan
                kurt = np.nan
        min_value = values.min()
        max_value = values.max()
        range = max_value - min_value
        p95 = values.quantile(0.95)
        p5 = values.quantile(0.05)
        p99 = values.quantile(0.99)
        p1 = values.quantile(0.01)
        upper_tail_diff = p99 - p95
        lower_tail_diff = p5 - p1
        tail_length_95 = max_value - p95
        tail_length_99 = max_value - p99
        tail_length_05 = p5 - min_value 
        tail_length_01 = p1 - min_value
        upper_tail = values[values >= values.quantile(0.95)]
        lower_tail = values[values <= values.quantile(0.05)]
        if len(unique_values) <= 2: # discrete distribution?
            upper_tail_mean = 0
            upper_tail_var = 0
            lower_tail_mean = 0
            lower_tail_var = 0
        else: 
            upper_tail_mean = upper_tail.mean()
            upper_tail_var = upper_tail.var()
            lower_tail_mean = lower_tail.mean()
            lower_tail_var = lower_tail.var()
        #percentile_ratio = p95 / p5 if p5 != 0 else np.nan
        tail_weight_ratio = np.sum((values > (median + 1.5*iqr)) | (values < (median - 1.5*iqr))) / len(values)
        excess_kurtosis = kurt - 3

        # outlier detection
        zscores = zscore(values)
        outliers_zscore = zscores.abs() > 3 
        count_outliers_zscore = outliers_zscore.sum() # using zscore
        Q1 = values.quantile(0.25)
        Q3 = values.quantile(0.75)
        IQR = Q3 - Q1   
        outliers_iqr = ((values < (Q1 - 1.5 * IQR)) | values > (Q3 + 1.5 * IQR)) # using iqr
        count_outliers_iqr = outliers_iqr.sum()
        # if len(values) > 1:
        #     lof = LocalOutlierFactor(n_neighbors=min(20, len(values) - 1)) # using local density of points
        #     lof_scores = lof.fit_predict(values.values.reshape(-1, 1))
        #     count_outliers_lof = np.sum(lof_scores == -1)
        # else:
        #     count_outliers_lof = 0
        # goodness of fit parameters
        warnings.simplefilter("ignore", category=RuntimeWarning)
        fitted_mean, fitted_std_dev = norm.fit(values)
        ks_stat_norm, ks_p_value_norm = kstest(values, 'norm', args=(fitted_mean, fitted_std_dev))
        warnings.filterwarnings("ignore", message="scipy.stats.shapiro: For N > 5000")
        if len(values) >= 3 and np.ptp(values) > 0:
            shapiro_stat, shapiro_p_value = shapiro(values)
        else:
            shapiro_stat, shapiro_p_value = np.nan, np.nan #float('nan'), float('nan')

        feature_vector = {
                'Mean': mean,
                'Median': median,
                'Std_Dev': std_dev,
                'IQR': iqr,
                'Skewness': skewness,
                'Kurtosis': kurt,
                'Min': min_value,
                'Max': max_value,
                'Range': range,
                'Upper_Tail': upper_tail_diff,
                'Lower_Tail': lower_tail_diff,
                'Extreme_Tail_95': tail_length_95,
                'Extreme_Tail_99': tail_length_99,
                'Extreme_Tail_05': tail_length_05,
                'Extreme_Tail_01': tail_length_01,
                'Upper_Tail_Mean': upper_tail_mean,
                'Upper_Tail_Var': upper_tail_var,
                'Lower_Tail_Mean': lower_tail_mean,
                'Lower_Tail_Mean': lower_tail_var,
                #'Percentile_Ratio_95_5': percentile_ratio,
                'Tail_Weight_Ratio': tail_weight_ratio,
                'Excess_Kurtosis': excess_kurtosis,
                'P99': p99,
                'P1': p1,
                'Outliers_Zscore': count_outliers_zscore,
                'Outliers_IQR': count_outliers_iqr,
                #'Outliers_LOF': count_outliers_lof,
                'KS_Stat_norm': ks_stat_norm,
                'KS_P_value_norm': ks_p_value_norm,
                'Shapiro_Stat': shapiro_stat,
                'Shapiro_P_value': shapiro_p_value
        }
        feature_vectors.append(feature_vector)

    features_df = pd.DataFrame(feature_vectors)
    return features_df

In [17]:
# applying function on the features 
ty_features = all_features(scaled_features_ty)
ty_features.head()

,Mean,Median,Std_Dev,IQR,Skewness,Kurtosis,Min,Max,Range,Upper_Tail,...,Tail_Weight_Ratio,Excess_Kurtosis,P99,P1,Outliers_Zscore,Outliers_IQR,KS_Stat_norm,KS_P_value_norm,Shapiro_Stat,Shapiro_P_value
0,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,-3.000000,0.000000,0.000000,0,0,NaN,NaN,NaN,NaN
1,8.692199e-16,-0.384006,1.000006,0.495033,2.765450e+00,19.370179,-1.536323,10.604948,12.141271,0.147351,...,0.254540,16.370179,1.972504,-1.400562,128,79783,0.288517,0.000000e+00,0.716606,5.274157e-137
2,8.549704e-17,0.000065,1.000006,1.732420,5.386640e-08,-1.200000,-1.731747,1.731877,3.463624,0.138594,...,0.000000,-4.200000,1.697229,-1.697208,0,0,0.057294,4.318635e-228,0.954930,4.079635e-87
3,-1.807265e-14,0.002162,1.000006,0.867089,-2.258166e+01,2952.347830,-80.369501,39.794400,120.163901,0.199185,...,0.006919,2949.347830,1.073987,-1.227523,57,0,0.142341,0.000000e+00,0.379675,9.769087e-162
4,-1.807265e-14,0.002162,1.000006,0.867089,-2.258166e+01,2952.347830,-80.369501,39.794400,120.163901,0.199185,...,0.006919,2949.347830,1.073987,-1.227523,57,0,0.142341,0.000000e+00,0.379675,9.769087e-162


# Combining feature space with target labels

In [18]:
# Combining the final features with the target labels
df_final_ty = pd.concat([labels_ty.T, ty_features], axis=1)
df_final_ty

,Name,Distribution Type,Mean,Median,Std_Dev,IQR,Skewness,Kurtosis,Min,Max,...,Tail_Weight_Ratio,Excess_Kurtosis,P99,P1,Outliers_Zscore,Outliers_IQR,KS_Stat_norm,KS_P_value_norm,Shapiro_Stat,Shapiro_P_value
0,temperature,functional,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,...,0.000000,-3.000000,0.000000,0.000000,0,0,NaN,NaN,NaN,NaN
1,Device_Test_Time,bimodal,8.692199e-16,-0.384006,1.000006,0.495033,2.765450e+00,19.370179,-1.536323,10.604948,...,0.254540,16.370179,1.972504,-1.400562,128,79783,0.288517,0.000000e+00,0.716606,5.274157e-137
2,Part_Id,bimodal,8.549704e-17,0.000065,1.000006,1.732420,5.386640e-08,-1.200000,-1.731747,1.731877,...,0.000000,-4.200000,1.697229,-1.697208,0,0,0.057294,4.318635e-228,0.954930,4.079635e-87
3,a_openXv_SIG_x_x_500uA__NFC_GPIO0,outlier,-1.807265e-14,0.002162,1.000006,0.867089,-2.258166e+01,2952.347830,-80.369501,39.794400,...,0.006919,2949.347830,1.073987,-1.227523,57,0,0.142341,0.000000e+00,0.379675,9.769087e-162
4,a_shortXv_SIG_x_x_500uA__NFC_GPIO0,outlier,-1.807265e-14,0.002162,1.000006,0.867089,-2.258166e+01,2952.347830,-80.369501,39.794400,...,0.006919,2949.347830,1.073987,-1.227523,57,0,0.142341,0.000000e+00,0.379675,9.769087e-162
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1397,f_nvm_programXxXPgmPatrepairinfo2M1YvhsYvhsYyY...,functional,1.660147e-16,-0.162416,1.000390,0.000000,0.000000e+00,0.000000,-0.162416,6.157036,...,0.025701,-3.000000,6.157036,-0.162416,33,1284,0.538810,0.000000e+00,0.142840,4.190848e-60
1398,f_nvm_readifrXxXVfyPatrepairinfo2M1YvhsYvhsYyY...,functional,1.660147e-16,-0.162416,1.000390,0.000000,0.000000e+00,0.000000,-0.162416,6.157036,...,0.025701,-3.000000,6.157036,-0.162416,33,1284,0.538810,0.000000e+00,0.142840,4.190848e-60
1399,f_nvm_XxXRdoutPatrepairinfo3M1YvhsYvhsYyYYy_1....,functional,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,...,0.000000,-3.000000,0.000000,0.000000,0,0,NaN,NaN,NaN,NaN
1400,f_nvm_readifrXxXVfyPatrepairinfo3M1YvhsYvhsYyY...,functional,-2.293553e-18,0.839089,1.000161,2.030858,0.000000e+00,0.000000,-1.191769,0.839089,...,0.000000,-3.000000,0.839089,-1.191769,0,0,0.386121,0.000000e+00,0.625566,1.923310e-63


# Binary re-labelling (converting to binary classification)

In [19]:
# function to relabel target variable into binary values, where outlier and longtail are grouped together
def relabel_data(original_label):
    if original_label in ['outlier', 'longtail']:  
        return 'Outlier/Longtail'
    else:
        return 'Other'

In [20]:
# new column added with the updated labels 
df_final_ty['Target'] = df_final_ty['Distribution Type'].apply(relabel_data)

# Encoding binary label into numerical target values

In [21]:
# encodiing target label into binary variable
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_encoded_test = label_encoder.fit_transform(df_final_ty['Target'])
df_final_ty['Target_Encoded'] = y_encoded_test

In [22]:
# sorting the columns for easier viewing of target labels
reordered_columns = ['Name'] + ['Target'] + ['Target_Encoded'] + [col for col in df_final_ty.columns if col not in ['Target', 'Name', 'Target_Encoded']]
df_final_ty = df_final_ty[reordered_columns]
df_final_ty

,Name,Target,Target_Encoded,Distribution Type,Mean,Median,Std_Dev,IQR,Skewness,Kurtosis,...,Tail_Weight_Ratio,Excess_Kurtosis,P99,P1,Outliers_Zscore,Outliers_IQR,KS_Stat_norm,KS_P_value_norm,Shapiro_Stat,Shapiro_P_value
0,temperature,Other,0,functional,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,...,0.000000,-3.000000,0.000000,0.000000,0,0,NaN,NaN,NaN,NaN
1,Device_Test_Time,Other,0,bimodal,8.692199e-16,-0.384006,1.000006,0.495033,2.765450e+00,19.370179,...,0.254540,16.370179,1.972504,-1.400562,128,79783,0.288517,0.000000e+00,0.716606,5.274157e-137
2,Part_Id,Other,0,bimodal,8.549704e-17,0.000065,1.000006,1.732420,5.386640e-08,-1.200000,...,0.000000,-4.200000,1.697229,-1.697208,0,0,0.057294,4.318635e-228,0.954930,4.079635e-87
3,a_openXv_SIG_x_x_500uA__NFC_GPIO0,Outlier/Longtail,1,outlier,-1.807265e-14,0.002162,1.000006,0.867089,-2.258166e+01,2952.347830,...,0.006919,2949.347830,1.073987,-1.227523,57,0,0.142341,0.000000e+00,0.379675,9.769087e-162
4,a_shortXv_SIG_x_x_500uA__NFC_GPIO0,Outlier/Longtail,1,outlier,-1.807265e-14,0.002162,1.000006,0.867089,-2.258166e+01,2952.347830,...,0.006919,2949.347830,1.073987,-1.227523,57,0,0.142341,0.000000e+00,0.379675,9.769087e-162
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1397,f_nvm_programXxXPgmPatrepairinfo2M1YvhsYvhsYyY...,Other,0,functional,1.660147e-16,-0.162416,1.000390,0.000000,0.000000e+00,0.000000,...,0.025701,-3.000000,6.157036,-0.162416,33,1284,0.538810,0.000000e+00,0.142840,4.190848e-60
1398,f_nvm_readifrXxXVfyPatrepairinfo2M1YvhsYvhsYyY...,Other,0,functional,1.660147e-16,-0.162416,1.000390,0.000000,0.000000e+00,0.000000,...,0.025701,-3.000000,6.157036,-0.162416,33,1284,0.538810,0.000000e+00,0.142840,4.190848e-60
1399,f_nvm_XxXRdoutPatrepairinfo3M1YvhsYvhsYyYYy_1....,Other,0,functional,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,...,0.000000,-3.000000,0.000000,0.000000,0,0,NaN,NaN,NaN,NaN
1400,f_nvm_readifrXxXVfyPatrepairinfo3M1YvhsYvhsYyY...,Other,0,functional,-2.293553e-18,0.839089,1.000161,2.030858,0.000000e+00,0.000000,...,0.000000,-3.000000,0.839089,-1.191769,0,0,0.386121,0.000000e+00,0.625566,1.923310e-63


In [23]:
binary_distribution_counts = df_final_ty['Target'].value_counts()
binary_distribution_counts

Target
Other               1239
Outlier/Longtail     163
Name: count, dtype: int64

# Checking for null statistical features

In [24]:
# filter rows where any column has null values
null_df = df_final_ty[df_final_ty.isnull().any(axis=1)]

# check the target label (distribution) of these null rows
null_df['Target'].value_counts()

null_df.isnull().sum()

Name                   0
Target                 0
Target_Encoded         0
Distribution Type      0
Mean                   0
Median                 0
Std_Dev                0
IQR                    0
Skewness               0
Kurtosis               0
Min                    0
Max                    0
Range                  0
Upper_Tail             0
Lower_Tail             0
Extreme_Tail_95        0
Extreme_Tail_99        0
Extreme_Tail_05        0
Extreme_Tail_01        0
Upper_Tail_Mean        0
Upper_Tail_Var         0
Lower_Tail_Mean        0
Tail_Weight_Ratio      0
Excess_Kurtosis        0
P99                    0
P1                     0
Outliers_Zscore        0
Outliers_IQR           0
KS_Stat_norm         263
KS_P_value_norm      263
Shapiro_Stat         263
Shapiro_P_value      263
dtype: int64

# Dropping rows with any null values

In [25]:
# drop those rows (tests) with any null values for more consistency and since all are of the negative class 
df_final_ty_nonull = df_final_ty.dropna()
df_final_ty_nonull

,Name,Target,Target_Encoded,Distribution Type,Mean,Median,Std_Dev,IQR,Skewness,Kurtosis,...,Tail_Weight_Ratio,Excess_Kurtosis,P99,P1,Outliers_Zscore,Outliers_IQR,KS_Stat_norm,KS_P_value_norm,Shapiro_Stat,Shapiro_P_value
1,Device_Test_Time,Other,0,bimodal,8.692199e-16,-0.384006,1.000006,0.495033,2.765450e+00,19.370179,...,0.254540,16.370179,1.972504,-1.400562,128,79783,0.288517,0.000000e+00,0.716606,5.274157e-137
2,Part_Id,Other,0,bimodal,8.549704e-17,0.000065,1.000006,1.732420,5.386640e-08,-1.200000,...,0.000000,-4.200000,1.697229,-1.697208,0,0,0.057294,4.318635e-228,0.954930,4.079635e-87
3,a_openXv_SIG_x_x_500uA__NFC_GPIO0,Outlier/Longtail,1,outlier,-1.807265e-14,0.002162,1.000006,0.867089,-2.258166e+01,2952.347830,...,0.006919,2949.347830,1.073987,-1.227523,57,0,0.142341,0.000000e+00,0.379675,9.769087e-162
4,a_shortXv_SIG_x_x_500uA__NFC_GPIO0,Outlier/Longtail,1,outlier,-1.807265e-14,0.002162,1.000006,0.867089,-2.258166e+01,2952.347830,...,0.006919,2949.347830,1.073987,-1.227523,57,0,0.142341,0.000000e+00,0.379675,9.769087e-162
5,a_openXv_SIG_x_x_500uA__NFC_GPIO1,Outlier/Longtail,1,outlier,-3.491200e-13,0.004359,1.000006,0.869917,-2.334583e+01,3021.331595,...,0.006906,3018.331595,1.073031,-1.229605,57,0,0.141582,0.000000e+00,0.380297,1.053992e-161
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1396,f_nvm_readifrXxXVfyPatrepairinfo2M1YvhsYvhsYyY...,Other,0,functional,-9.953480e-17,0.831527,1.000159,2.034134,0.000000e+00,0.000000,...,0.000000,-3.000000,0.831527,-1.202606,0,0,0.388375,0.000000e+00,0.624411,8.568693e-64
1397,f_nvm_programXxXPgmPatrepairinfo2M1YvhsYvhsYyY...,Other,0,functional,1.660147e-16,-0.162416,1.000390,0.000000,0.000000e+00,0.000000,...,0.025701,-3.000000,6.157036,-0.162416,33,1284,0.538810,0.000000e+00,0.142840,4.190848e-60
1398,f_nvm_readifrXxXVfyPatrepairinfo2M1YvhsYvhsYyY...,Other,0,functional,1.660147e-16,-0.162416,1.000390,0.000000,0.000000e+00,0.000000,...,0.025701,-3.000000,6.157036,-0.162416,33,1284,0.538810,0.000000e+00,0.142840,4.190848e-60
1400,f_nvm_readifrXxXVfyPatrepairinfo3M1YvhsYvhsYyY...,Other,0,functional,-2.293553e-18,0.839089,1.000161,2.030858,0.000000e+00,0.000000,...,0.000000,-3.000000,0.839089,-1.191769,0,0,0.386121,0.000000e+00,0.625566,1.923310e-63


# Saving processed data files
one dataset with null values (full), one set without null values

In [26]:
import pickle

data = {'ty_full': df_final_ty, 'ty_dropped': df_final_ty_nonull}

with open('ty_dataframes.pkl', 'wb') as file:
    pickle.dump(data, file)